In [5]:
import pandas as pd
import itertools
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
from sql import get_data

In [5]:
df = pd.DataFrame()
df['keyword'] = ['haftpflichtversicherung private', 'private haftpflichtversicherung', 'vermögensschaden haftpflicht', 'privat haftpflicht versicherung', 'privat haftpflichtversicherung', 'privat haftpflicht', 'frachtführerhaftpflichtversicherung', 'bayerische haftpflicht', 'haftpflichtversicherung haus', 'bauherrenhaftpflicht', 'bauherren haftpflichtversicherung', 'haus und grundbesitzer haftpflichtversicherung', 'haus haftpflicht', 'haus und grundbesitzer haftpflicht', 'grundbesitzer haftpflicht', 'haus grundbesitzer haftpflicht', 'versicherungskammer bayern haftpflicht', 'schadenfreiheitsklasse haftpflicht', 'schadenfreiheitsklassen haftpflicht', 'haftpflichtschaden melden']
df['clicks_per_mo'] = [29.5, 740.0, 36.0, 405.0, 405.0, 330.0, 8.5, 5.5, 13.0, 220.0, 80.0, 44.0, 5.5, 145.0, 65.0, 7.0, 7.0, 7.0, 7.0, 36.0]
df['costs_per_mo'] = [298.84, 6127.2, 270.36, 2818.8, 2818.8, 2115.3, 53.97, 25.58, 49.66, 814.0, 292.0, 107.36, 12.32, 287.1, 115.05, 12.04, 10.08, 7.35, 7.35, 36.36]
df.head(20)

,keyword,clicks_per_mo,costs_per_mo
0,haftpflichtversicherung private,29.5,298.84
1,private haftpflichtversicherung,740.0,6127.20
2,vermögensschaden haftpflicht,36.0,270.36
3,privat haftpflicht versicherung,405.0,2818.80
4,privat haftpflichtversicherung,405.0,2818.80
5,privat haftpflicht,330.0,2115.30
6,frachtführerhaftpflichtversicherung,8.5,53.97
7,bayerische haftpflicht,5.5,25.58
8,haftpflichtversicherung haus,13.0,49.66
9,bauherrenhaftpflicht,220.0,814.00


In [6]:
# read test data from sql
df = get_data('SELECT * FROM keywords_test')

# define percentage of clicks from ads_search_volume
estimated_percentage_clicks = 0.05
df['clicks_per_mo'] = df['avg_monthly_searches'] * estimated_percentage_clicks 
df['costs_per_mo'] = df['avg_monthly_searches'] * estimated_percentage_clicks * df['suggested_bid']

df.head()

,keyword,avg_monthly_searches,competition,platform,suggested_bid,clicks_per_mo,costs_per_mo
0,1.27 mm pitch ribbon cable,10.0,None,google,NaN,0.5,NaN
1,10 conductor ribbon cable,0.0,None,google,NaN,0.0,NaN
2,10 pin flat cable,10.0,None,google,NaN,0.5,NaN
3,10 pin flat ribbon cable,10.0,None,google,NaN,0.5,NaN
4,12v dc dc converter,140.0,Hoch,google,0.73,7.0,5.11


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1875 entries, 0 to 1874
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   keyword               1875 non-null   object 
 1   avg_monthly_searches  1675 non-null   float64
 2   competition           1324 non-null   object 
 3   platform              1875 non-null   object 
 4   suggested_bid         1030 non-null   float64
 5   clicks_per_mo         1675 non-null   float64
 6   costs_per_mo          1030 non-null   float64
dtypes: float64(4), object(3)
memory usage: 102.7+ KB


## Optimization functions

In [4]:
def clicks_max_level1(df, budget):
    """
    This function calculates the best combination of keywords by maximizing the clicks.
    It sorts the df by clicks per mo descending, takes the first keyword, 
    then go to the next keyword and take it, if we can afford (total sum of taken keywords not exceeds the budget),
    next keywords as well
    """

    # filter dataframe for keywords with lower costs than budget
    df = df[df['costs_per_mo']<=budget]
    
    # sort value by klicks descending and costs ascending
    df = df.sort_values(['clicks_per_mo', 'costs_per_mo'], ascending=[False, True]).reset_index(drop=True)
    
    # define variables: empty list for the index of the optimal keywords; running sum of costs for optimal keywords
    key_position = []
    costs = 0.0

    # loop through the filtered dataframe
    for idx, row in df.iterrows():
        # take the first row 
        if idx == 0:
            key_position.append(idx)
            costs += row['costs_per_mo']
        # add costs of the additional keyword while running sum of budget is less then budget
        elif costs + row['costs_per_mo'] <= budget:
            key_position.append(idx)
            costs += row['costs_per_mo']

    # return total costs, klicks of best combination of similar keywords
    
    return df.iloc[key_position].reset_index(drop=True)

In [5]:
def clicks_max_level2(df, budget):
    """
    This function calculates the best combination of keywords by maximizing the clicks.
    It searchs for the optimal combination of keywords by iterating through all possible combination 
    saving the maximum of clicks at a given budget.
    """
    
    # check if you can take all keywords with the given budget
    if df.costs_per_mo.sum() <= budget:
        return print(f'You need a budget of {round(df.costs_per_mo.sum(),2)} Euro to buy all the keywords!')

    # exclude all keywords that exceed the budget
    df = df[df['costs_per_mo'] <= budget]
    
    # create iterator for all possible combination (True if to choose the keyword, false if not)
    comb = itertools.product([True,False], repeat=len(list(df.keyword)))

    comb_max = []
    clicks = 0

    # loop through all combinations
    for x in comb:
        cl, co = df[list(x)][['clicks_per_mo', 'costs_per_mo']].sum()

        # save the combination if its better than previous saved combination
        if (cl > clicks) and (co <= budget):
            comb_max = list(x)
            clicks = cl

    # return dataframe with the optimal keywords combination
    return df[comb_max].reset_index(drop=True)

## Compare results

In [6]:
# run the first function
opt1 = clicks_max_level1(df.loc[:14], 3000)
opt1[['clicks_per_mo', 'costs_per_mo']].sum()

clicks_per_mo     488.50
costs_per_mo     2995.83
dtype: float64

In [7]:
# run the second function
opt2 = clicks_max_level2(df.loc[:14], 3000)
opt2[['clicks_per_mo', 'costs_per_mo']].sum()

clicks_per_mo     682.50
costs_per_mo     2992.05
dtype: float64

So with the second function we get better results. But if we raise the number of keywords this function takes to long.

In [8]:
opt1

,keyword,clicks_per_mo,costs_per_mo
0,privat haftpflicht versicherung,405.0,2818.80
1,grundbesitzer haftpflicht,65.0,115.05
2,haftpflichtversicherung haus,13.0,49.66
3,haus haftpflicht,5.5,12.32


In [9]:
opt2

,keyword,clicks_per_mo,costs_per_mo
0,privat haftpflicht,330.0,2115.30
1,bayerische haftpflicht,5.5,25.58
2,haftpflichtversicherung haus,13.0,49.66
3,bauherren haftpflichtversicherung,80.0,292.00
4,haus und grundbesitzer haftpflichtversicherung,44.0,107.36
5,haus und grundbesitzer haftpflicht,145.0,287.10
6,grundbesitzer haftpflicht,65.0,115.05


We are now searching for a more advanced and fast way to get the maximum of clicks at a certain budget.

## Advanced optimization

In [8]:
df.head(20)

,keyword,clicks_per_mo,costs_per_mo
0,haftpflichtversicherung private,29.5,298.84
1,private haftpflichtversicherung,740.0,6127.20
2,vermögensschaden haftpflicht,36.0,270.36
3,privat haftpflicht versicherung,405.0,2818.80
4,privat haftpflichtversicherung,405.0,2818.80
5,privat haftpflicht,330.0,2115.30
6,frachtführerhaftpflichtversicherung,8.5,53.97
7,bayerische haftpflicht,5.5,25.58
8,haftpflichtversicherung haus,13.0,49.66
9,bauherrenhaftpflicht,220.0,814.00


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   keyword        20 non-null     object 
 1   clicks_per_mo  20 non-null     float64
 2   costs_per_mo   20 non-null     float64
dtypes: float64(2), object(1)
memory usage: 608.0+ bytes


In [8]:
def clicks_max_level3(df, budget):
    """
    This function calculates the best combination of keywords by maximizing the clicks using a linear solver.
    """
    
    # check if you can take all keywords with the given budget
    if df.costs_per_mo.sum() <= budget:
        return print(f'You need a budget of {round(df.costs_per_mo.sum(),2)} Euro to buy all the keywords!')

    # exclude all keywords that exceed the budget
    df = df[df['costs_per_mo'] <= budget].reset_index(drop=True)
    
    indici = list(df.index.values)
    clicks = df['clicks_per_mo'].values
    costs = df['costs_per_mo'].values

    # create a concrete model
    model = pyo.ConcreteModel()

    # define the VARIABLES (in the end 0=not selected, 1=selected)
    model.x = pyo.Var(indici, within=pyo.Binary)
    x = model.x

    # define the CONSTRAINT, the total costs should be less than budget
    model.weight_constraint = pyo.Constraint(expr= sum([x[p]*costs[p] for p in indici]) <= budget)

    # define the OBJECTIVE, we want to maximize the value of the selected keywords
    model.objective = pyo.Objective(expr= sum([x[p]*clicks[p] for p in indici]), sense=pyo.maximize)

    # print the complete model
    #model.pprint()

    # call the solver
    # replace the executable path by your own
    opt = SolverFactory('cbc', executable='cbc-osx/cbc')
    results = opt.solve(model)

    solution = [int(pyo.value(model.x[p])) for p in indici]
    solution = [bool(x) for x in solution]

    # return dataframe with the optimal keywords combination
    return df[solution].reset_index(drop=True)

In [9]:
# run the third function
opt3 = clicks_max_level3(df, 3000)
opt3[['clicks_per_mo', 'costs_per_mo']].sum()

clicks_per_mo    7831.25600
costs_per_mo     2999.99815
dtype: float64

Optimization with linear programming step by step

In [38]:
budget = 3000.0
df1 = df[df['costs_per_mo'] <= budget].reset_index(drop=True)
df1

,keyword,clicks_per_mo,costs_per_mo
0,haftpflichtversicherung private,29.5,298.84
1,vermögensschaden haftpflicht,36.0,270.36
2,privat haftpflicht versicherung,405.0,2818.80
3,privat haftpflichtversicherung,405.0,2818.80
4,privat haftpflicht,330.0,2115.30
5,frachtführerhaftpflichtversicherung,8.5,53.97
6,bayerische haftpflicht,5.5,25.58
7,haftpflichtversicherung haus,13.0,49.66
8,bauherrenhaftpflicht,220.0,814.00
9,bauherren haftpflichtversicherung,80.0,292.00


In [70]:
indici = list(df1.index.values)
clicks = df1['clicks_per_mo'].values
costs = df1['costs_per_mo'].values

In [60]:
clicks

array([ 29.5,  36. , 405. , 405. , 330. ,   8.5,   5.5,  13. , 220. ,
        80. ,  44. ,   5.5, 145. ,  65. ,   7. ,   7. ,   7. ,   7. ,
        36. ])

In [71]:
# create a concrete model
model = pyo.ConcreteModel()

# define the VARIABLES (in the end 0=not selected, 1=selected)
model.x = pyo.Var(indici, within=pyo.Binary)
x = model.x

# define the CONSTRAINT, the total costs should be less than budget
model.weight_constraint = pyo.Constraint(expr= sum([x[p]*costs[p] for p in indici]) <= budget)

# define the OBJECTIVE, we want to maximize the value of the selected keywords
model.objective = pyo.Objective(expr= sum([x[p]*clicks[p] for p in indici]), sense=pyo.maximize)

# print the complete model
model.pprint()

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   19 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18}

1 Var Declarations
    x : Size=19, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 :     0 :  None :     1 : False :  True : Binary
          9 :     0 :  None :     1 : False :  True : Binary
         10 :     0 :  None :     1 : False :  True : 

In [77]:
# replace the executable path by your own
opt = SolverFactory('cbc', executable='/Users/damjan/Downloads/cbc-osx/cbc')
results = opt.solve(model)

In [78]:
solution = [int(pyo.value(model.x[p])) for p in indici]
df1['solution'] = solution

In [79]:
df1

,keyword,clicks_per_mo,costs_per_mo,solution
0,haftpflichtversicherung private,29.5,298.84,0
1,vermögensschaden haftpflicht,36.0,270.36,0
2,privat haftpflicht versicherung,405.0,2818.80,0
3,privat haftpflichtversicherung,405.0,2818.80,0
4,privat haftpflicht,330.0,2115.30,1
5,frachtführerhaftpflichtversicherung,8.5,53.97,0
6,bayerische haftpflicht,5.5,25.58,0
7,haftpflichtversicherung haus,13.0,49.66,0
8,bauherrenhaftpflicht,220.0,814.00,0
9,bauherren haftpflichtversicherung,80.0,292.00,1


In [75]:
df1['solution'] = df1['solution'].replace({0:False, 1:True})

In [76]:
df1[df1.solution][['clicks_per_mo', 'costs_per_mo']].sum()

clicks_per_mo     728.00
costs_per_mo     2989.99
dtype: float64